涨停假阴线
涨停次日开盘价小于收盘价，且收盘价大于昨日涨停价
1. 连板数小于5板
2. 放量换手板
3. 涨停次日收假阴线
4. 假阴后股价能站在5日线上
5. 两日内突破假阴线
6. 突破假阴线上车，回踩5日可加码

In [1]:
import akshare as ak
import pandas as pd

In [32]:
code = '603366'
stock = ak.stock_zh_a_hist(symbol=code, 
                        period="daily",
                        start_date="20231001",
                        end_date='20250325', 
                        adjust="")

In [33]:
stock['MA5'] = stock['收盘'].rolling(window=5).mean()
stock['ZA5'] = stock['涨跌幅'].rolling(window=5).mean()
stock['MA10'] = stock['收盘'].rolling(window=10).mean()
stock['MA25'] = stock['收盘'].rolling(window=25).mean()
stock = stock[['日期','股票代码','开盘','收盘','最高','最低','成交量','振幅','涨跌幅','涨跌额','MA5','MA10','MA25','ZA5']]
stock.head(2)

,日期,股票代码,开盘,收盘,最高,最低,成交量,振幅,涨跌幅,涨跌额,MA5,MA10,MA25,ZA5
0,2023-10-09,603366,5.91,5.88,6.02,5.84,156077,3.03,-1.01,-0.06,NaN,NaN,NaN,NaN
1,2023-10-10,603366,5.88,5.74,5.92,5.72,127046,3.40,-2.38,-0.14,NaN,NaN,NaN,NaN


In [34]:
df = stock.copy()

In [38]:
# 入场买点
def buy(df):
    buy = []
    for i in range(30,len(df)-1):
        if all ([df['开盘'][i] > df['收盘'][i],\
                df['收盘'][i] > df['收盘'][i-1],\
                df['涨跌幅'][i-1] > 9.9,\
                df['ZA5'][i-1]>5,\
                df['ZA5'][i-1]<9.8]):
            if all([df['收盘'][i+1] > df['MA5'][i+1],\
                    df['收盘'][i+1] > df['开盘'][i]]):
                buy.append(i+1)
                break
            elif all([df['收盘'][i+1] > df['MA5'][i+1],\
                     df['收盘'][i+2] > df['MA5'][i+2],\
                     df['收盘'][i+2] > df['开盘'][i]]):
                buy.append(i+2)
                break
            else:
                continue
        else:
            continue
    return buy

In [39]:
def sale(b):
    b_high = df['最高'][b]*1.02 #盈利线
    b_low = b_high*0.9 #止损线
    for w in range(b+1,len(df)):
        # 收盘价达到盈利线
        if df['最高'][w]>b_high:
            s = w
            b_high = df['最高'][w]
            b_low =  b_low = b_high*0.9
            continue
        # 跌破止损价，卖出
        elif all([df['收盘'][w]<b_low,\
                 df['涨跌幅'][w]<-5,]):
            s = w
            break
        else:
            s = w
            continue
    return s  

In [40]:
# 交易回测
money = 10000
b_dt = buy(df)
print(b_dt)
if not b_dt:
    print('无符合条件指标，请更换股票')
else:
    s_date = df['日期'][1]
    for k in range(len(b_dt)):
        b = b_dt[k]
        if df['日期'][b]>s_date:
            #print(str(df['日期'][b]) + ' | ' + str(df['CCI'][b]))
            b_sym = float(((money/df['收盘'][b])//100)*100)
            money = money-(b_sym*df['收盘'][b])
            print('买入： '+ str(df['日期'][b]) + ' | ' + str(df['收盘'][b]) + ' | ' + str(b_sym) + ' | ' + str(money))
            s = sale(b)
            money = money+(b_sym*df['收盘'][s])
            b_sym = 0
            s_date = df['日期'][s]
            print('买出： '+ str(df['日期'][s]) + ' | ' + str(df['收盘'][s]) + ' | '+ str(b_sym) + '     | ' + str(money))
        else:
            continue

[279]
买入： 2024-11-29 | 21.78 | 400.0 | 1288.0
买出： 2024-12-03 | 18.01 | 0     | 8492.0


##### 